# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier

In [3]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', 'sqlite:///InsertDatabaseName.db')
X = df.message.values
Y = df[df.columns[4:]].values
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [36]:
def tokenize(text):
    # Todo: normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Todo: tokenize text
    words = word_tokenize(text)
    
    # Todo: lemmatize and remove stop words
    words = [w for w in words if w not in stopwords.words("english")]
    tokens = [WordNetLemmatizer().lemmatize(w) for w in words]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [58]:
pipeline = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators = 10, n_jobs=-1)))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [76]:
from sklearn.metrics import classification_report
y_pred = pipeline.predict(X_test)
columns = list(df.columns[4:])
#print(columns)
for i in range(36):
    print(columns[i])
    print(classification_report(y_test[:,i], y_pred[:,i]))

related
             precision    recall  f1-score   support

          0       0.61      0.35      0.45      1563
          1       0.82      0.93      0.87      4944
          2       0.57      0.45      0.50        47

avg / total       0.76      0.79      0.76      6554

request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5443
          1       0.83      0.40      0.54      1111

avg / total       0.88      0.88      0.87      6554

offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6521
          1       0.00      0.00      0.00        33

avg / total       0.99      0.99      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.73      0.88      0.80      3884
          1       0.75      0.52      0.61      2670

avg / total       0.74      0.73      0.72      6554

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [82]:
from sklearn.model_selection import GridSearchCV
pipeline.get_params()

parameters =  {
        'clf__estimator__n_estimators': [10],
        'clf__estimator__min_samples_split': [2, 4]}

cv = GridSearchCV(pipeline, param_grid=parameters)

cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__n_estimators': [10], 'clf__estimator__min_samples_split': [2, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [83]:
y_pred = cv.predict(X_test)

for i in range(36):
    print(columns[i])
    print(classification_report(y_test[:,i], y_pred[:,i]))

related
             precision    recall  f1-score   support

          0       0.62      0.35      0.45      1563
          1       0.82      0.93      0.87      4944
          2       0.62      0.53      0.57        47

avg / total       0.77      0.79      0.77      6554

request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5443
          1       0.83      0.39      0.53      1111

avg / total       0.88      0.88      0.87      6554

offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6521
          1       0.00      0.00      0.00        33

avg / total       0.99      0.99      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.72      0.87      0.79      3884
          1       0.73      0.51      0.60      2670

avg / total       0.72      0.72      0.71      6554

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [104]:
def build_rforest():
    # Random Forest Classification
    pipeline = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators = 10, n_jobs=-1)))])
    parameters =  {
        'clf__estimator__n_estimators': [10, 15],
        'clf__estimator__min_samples_split': [2, 4]}
    
    # create grid search object
    cv = GridSearchCV(pipeline, param_grid=parameters, cv=2, verbose=2)
    
    return cv

def build_adaboost():
    # AdaBoostClassification
    pipeline = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultiOutputClassifier(AdaBoostClassifier(n_estimators = 10)))])
    parameters =  {
        'clf__estimator__n_estimators': [10],
        'clf__estimator__learning_rate': [0.1, 1]}
    
    # create grid search object
    cv = GridSearchCV(pipeline, param_grid=parameters, cv=2, verbose=2)
    
    return cv

model_rf = build_rforest()
model_rf.fit(X_train, y_train)

model_ada = build_adaboost()
model_ada.fit(X_train, y_train)


Fitting 2 folds for each of 4 candidates, totalling 8 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, total=  23.3s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   25.6s remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, total=  23.7s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=15 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=15, total=  34.2s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=15 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=15, total=  35.7s
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10 
[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10, total=  20.3s
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10 
[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10, total=  20.7s
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=15 
[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=15, total=  30.2s
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=15 
[CV]  clf_

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  4.0min finished


Fitting 2 folds for each of 2 candidates, totalling 4 fits
[CV] clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=10 
[CV]  clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=10, total=  17.9s
[CV] clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=10 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   19.2s remaining:    0.0s


[CV]  clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=10, total=  17.8s
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=10 
[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=10, total=  18.2s
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=10 
[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=10, total=  18.1s


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  1.3min finished


GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=10, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__n_estimators': [10], 'clf__estimator__learning_rate': [0.1, 1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [105]:
y_pred_rf = model_rf.predict(X_test)

for i in range(36):
    print(columns[i])
    print(classification_report(y_test[:,i], y_pred_rf[:,i]))

related
             precision    recall  f1-score   support

          0       0.67      0.27      0.38      1563
          1       0.80      0.96      0.87      4944
          2       0.71      0.51      0.59        47

avg / total       0.77      0.79      0.75      6554

request
             precision    recall  f1-score   support

          0       0.90      0.98      0.94      5443
          1       0.82      0.46      0.59      1111

avg / total       0.89      0.89      0.88      6554

offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6521
          1       0.00      0.00      0.00        33

avg / total       0.99      0.99      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.76      0.86      0.80      3884
          1       0.74      0.60      0.66      2670

avg / total       0.75      0.75      0.75      6554

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [106]:
y_pred_ada = model_ada.predict(X_test)

for i in range(36):
    print(columns[i])
    print(classification_report(y_test[:,i], y_pred_ada[:,i]))

related
             precision    recall  f1-score   support

          0       0.30      0.00      0.00      1563
          1       0.75      1.00      0.86      4944
          2       0.00      0.00      0.00        47

avg / total       0.64      0.75      0.65      6554

request
             precision    recall  f1-score   support

          0       0.90      0.97      0.93      5443
          1       0.76      0.47      0.58      1111

avg / total       0.88      0.89      0.87      6554

offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6521
          1       0.00      0.00      0.00        33

avg / total       0.99      0.99      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.72      0.85      0.78      3884
          1       0.71      0.53      0.60      2670

avg / total       0.72      0.72      0.71      6554

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

Result: Model with Random Forest Classifier performs better -> save model_rf with pickle

In [107]:
import pickle
pickle.dump(model_rf, open("model.pickle", 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.